# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

## Solution: Flight Booking Voice Agent

Audio-responsive attendant for a flight booking agency. Speak to search flights, get prices, and complete bookings using the **Amadeus API**.

**Optional:** Add keys to `.env` for live APIs (otherwise the app still runs; flight/voice features will prompt for keys when used):
- `OPENAI_API_KEY` (or `OPENROUTER_API_KEY` with OpenRouter) — Whisper, LLM, TTS
- `AMADEUS_CLIENT_ID` and `AMADEUS_CLIENT_SECRET` — flight search and booking
**Security:** Input validation (message length, IATA/date format, payload size), rate limiting (30 requests/minute).

In [ ]:
# imports
import os
import json
import tempfile
from datetime import datetime, timedelta
from typing import Any, Optional

import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv(override=True)

In [ ]:
# constants and environment (no auth check — keys from .env used when APIs are called)
MODEL = "gpt-5-nano"

openai_api_key = os.getenv("OPENROUTER_API_KEY")
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openai_api_key
)

In [ ]:
# Security: input validation, rate limiting, safe defaults
import re
import time

MAX_USER_MESSAGE_LENGTH = 4000
MAX_HISTORY_TURNS = 30
RATE_LIMIT_REQUESTS = 30
RATE_LIMIT_WINDOW_SECONDS = 60
MAX_TOOL_JSON_LENGTH = 50_000
IATA_PATTERN = re.compile(r"^[A-Za-z]{3}$")
DATE_PATTERN = re.compile(r"^\d{4}-\d{2}-\d{2}$")

_rate_limit_times = []

def _rate_limit_check():
    now = time.time()
    _rate_limit_times[:] = [t for t in _rate_limit_times if now - t < RATE_LIMIT_WINDOW_SECONDS]
    if len(_rate_limit_times) >= RATE_LIMIT_REQUESTS:
        return False
    _rate_limit_times.append(now)
    return True

def sanitize_user_input(text):
    if text is None:
        return ""
    s = str(text).replace("\x00", "").strip()
    return s[:MAX_USER_MESSAGE_LENGTH] if len(s) > MAX_USER_MESSAGE_LENGTH else s

def validate_iata(code):
    return code and isinstance(code, str) and bool(IATA_PATTERN.match(code.strip()))

def validate_date(s):
    return s and isinstance(s, str) and bool(DATE_PATTERN.match(s.strip()))

def validate_tool_args_search_flights(args):
    o = args.get("origin_location_code", "")
    d = args.get("destination_location_code", "")
    dt = args.get("departure_date", "")
    if not validate_iata(o): return "origin_location_code must be a 3-letter IATA code"
    if not validate_iata(d): return "destination_location_code must be a 3-letter IATA code"
    if not validate_date(dt): return "departure_date must be YYYY-MM-DD"
    adults = args.get("adults", 1)
    children = args.get("children", 0)
    if not isinstance(adults, int) or adults < 1 or adults > 9: return "adults must be 1-9"
    if not isinstance(children, int) or children < 0 or children > 9: return "children must be 0-9"
    rd = args.get("return_date")
    if rd is not None and not validate_date(rd): return "return_date must be YYYY-MM-DD"
    return None

def validate_tool_args_get_airports(args):
    kw = args.get("keyword", "")
    if not kw or not isinstance(kw, str) or len(kw.strip()) > 100:
        return "keyword must be a non-empty string up to 100 characters"
    return None

In [ ]:
# Amadeus API client (token cache + flight search, price confirm, booking)
AMADEUS_BASE = "https://test.api.amadeus.com"
AUTH_URL = f"{AMADEUS_BASE}/v1/security/oauth2/token"
FLIGHT_OFFERS_URL = f"{AMADEUS_BASE}/v2/shopping/flight-offers"
FLIGHT_OFFERS_PRICE_URL = f"{AMADEUS_BASE}/v1/shopping/flight-offers-pricing"
FLIGHT_CREATE_ORDERS_URL = f"{AMADEUS_BASE}/v1/booking/flight-orders"
LOCATIONS_URL = f"{AMADEUS_BASE}/v1/reference-data/locations"

class AmadeusClient:
    def __init__(self, client_id=None, client_secret=None):
        self.client_id = client_id or os.getenv("AMADEUS_CLIENT_ID")
        self.client_secret = client_secret or os.getenv("AMADEUS_CLIENT_SECRET")
        self._access_token = None
        self._token_expires_at = None
        self._buffer = 300

    def _get_token(self):
        if not self.client_id or not self.client_secret:
            return None
        r = requests.post(AUTH_URL, data={
            "grant_type": "client_credentials",
            "client_id": self.client_id,
            "client_secret": self.client_secret,
        }, headers={"Content-Type": "application/x-www-form-urlencoded"}, timeout=30)
        if r.status_code != 200:
            return None
        d = r.json()
        self._access_token = d.get("access_token")
        exp = d.get("expires_in", 1799) - self._buffer
        self._token_expires_at = datetime.now() + timedelta(seconds=exp)
        return self._access_token

    def _ensure_token(self):
        if self._access_token and self._token_expires_at and datetime.now() < self._token_expires_at:
            return True
        return self._get_token() is not None

    def _headers(self):
        return {"Authorization": f"Bearer {self._access_token}"} if self._ensure_token() else {}

    def search_flights(self, origin_location_code, destination_location_code, departure_date, return_date=None, adults=1, children=0):
        if not self._ensure_token():
            return {"error": "Unable to authenticate with Amadeus. Check credentials."}
        params = {"originLocationCode": origin_location_code.upper(), "destinationLocationCode": destination_location_code.upper(), "departureDate": departure_date, "adults": adults}
        if return_date:
            params["returnDate"] = return_date
        if children:
            params["children"] = children
        r = requests.get(FLIGHT_OFFERS_URL, params=params, headers=self._headers(), timeout=30)
        if r.status_code != 200:
            try:
                return {"error": r.json().get("errors", [{}])[0].get("detail", r.text)}
            except Exception:
                return {"error": r.text or "Flight search failed."}
        data = r.json()
        return {"data": data.get("data", []), "dictionaries": data.get("dictionaries", {})}

    def get_airports_or_cities(self, keyword, sub_type="AIRPORT"):
        if not self._ensure_token():
            return {"error": "Unable to authenticate with Amadeus. Check credentials."}
        r = requests.get(LOCATIONS_URL, params={"keyword": keyword, "subType": sub_type}, headers=self._headers(), timeout=30)
        if r.status_code != 200:
            try:
                return {"error": r.json().get("errors", [{}])[0].get("detail", r.text)}
            except Exception:
                return {"error": r.text or "Location search failed."}
        return {"data": r.json().get("data", [])}

    def confirm_flight_price(self, flight_offer):
        if not self._ensure_token():
            return {"error": "Unable to authenticate with Amadeus. Check credentials."}
        body = {"data": {"type": "flight-offers-pricing", "flightOffers": [flight_offer]}}
        r = requests.post(FLIGHT_OFFERS_PRICE_URL, json=body, headers={**self._headers(), "Content-Type": "application/json"}, timeout=30)
        if r.status_code != 200:
            try:
                return {"error": r.json().get("errors", [{}])[0].get("detail", r.text)}
            except Exception:
                return {"error": r.text or "Price confirmation failed."}
        return r.json()

    def create_flight_order(self, flight_offer, travelers):
        if not self._ensure_token():
            return {"error": "Unable to authenticate with Amadeus. Check credentials."}
        body = {"data": {"type": "flight-order", "flightOffers": [flight_offer], "travelers": travelers}}
        r = requests.post(FLIGHT_CREATE_ORDERS_URL, json=body, headers={**self._headers(), "Content-Type": "application/json"}, timeout=30)
        if r.status_code not in (200, 201):
            try:
                return {"error": r.json().get("errors", [{}])[0].get("detail", r.text)}
            except Exception:
                return {"error": r.text or "Booking failed."}
        return r.json()

amadeus = AmadeusClient()

In [ ]:
# OpenAI tool definitions for the flight agent
search_flights_tool = {
    "type": "function",
    "function": {
        "name": "search_flights",
        "description": "Search for flight offers between two airports. Use IATA codes (e.g. JFK, LHR). Call get_airports_or_cities first if user says a city name.",
        "parameters": {
            "type": "object",
            "properties": {
                "origin_location_code": {"type": "string", "description": "Origin airport IATA code"},
                "destination_location_code": {"type": "string", "description": "Destination airport IATA code"},
                "departure_date": {"type": "string", "description": "Departure date YYYY-MM-DD"},
                "return_date": {"type": "string", "description": "Return date YYYY-MM-DD (optional)"},
                "adults": {"type": "integer", "description": "Number of adults"},
                "children": {"type": "integer", "description": "Number of children"},
            },
            "required": ["origin_location_code", "destination_location_code", "departure_date"],
            "additionalProperties": False,
        },
    },
}

get_airports_tool = {
    "type": "function",
    "function": {
        "name": "get_airports_or_cities",
        "description": "Resolve a city or keyword to airport/city codes. Use before search_flights when user says a city name.",
        "parameters": {
            "type": "object",
            "properties": {
                "keyword": {"type": "string", "description": "City or place name"},
                "sub_type": {"type": "string", "description": "AIRPORT or CITY", "enum": ["AIRPORT", "CITY"]},
            },
            "required": ["keyword"],
            "additionalProperties": False,
        },
    },
}

confirm_price_tool = {
    "type": "function",
    "function": {
        "name": "confirm_flight_price",
        "description": "Confirm availability and final price for one flight offer (from search results). Pass the full offer object as JSON.",
        "parameters": {
            "type": "object",
            "properties": {
                "flight_offer_json": {"type": "string", "description": "JSON string of the selected flight offer from search_flights"},
            },
            "required": ["flight_offer_json"],
            "additionalProperties": False,
        },
    },
}

create_order_tool = {
    "type": "function",
    "function": {
        "name": "create_flight_order",
        "description": "Book a flight. Requires the flight offer (from search or price confirm) and traveler details: id, dateOfBirth (YYYY-MM-DD), name.firstName, name.lastName, gender (MALE/FEMALE).",
        "parameters": {
            "type": "object",
            "properties": {
                "flight_offer_json": {"type": "string", "description": "JSON string of the flight offer"},
                "travelers_json": {"type": "string", "description": "JSON array of travelers: [{\"id\": \"1\", \"dateOfBirth\": \"2000-01-01\", \"name\": {\"firstName\": \"JOHN\", \"lastName\": \"DOE\"}, \"gender\": \"MALE\"}]"},
            },
            "required": ["flight_offer_json", "travelers_json"],
            "additionalProperties": False,
        },
    },
}

TOOLS = [search_flights_tool, get_airports_tool, confirm_price_tool, create_order_tool]

def handle_tool_calls(message):
    responses = []
    for tc in message.tool_calls:
        name = tc.function.name
        try:
            args = json.loads(tc.function.arguments or "{}")
        except json.JSONDecodeError:
            responses.append({"role": "tool", "content": "Invalid arguments.", "tool_call_id": tc.id})
            continue
        if name == "search_flights":
            err = validate_tool_args_search_flights(args)
            if err:
                content = json.dumps({"error": err})
            else:
                
                out = amadeus.search_flights(
                    args.get("origin_location_code", "").strip().upper(),
                    args.get("destination_location_code", "").strip().upper(),
                    args.get("departure_date", "").strip(),
                    return_date=args.get("return_date", "").strip() or None,
                    adults=max(1, min(9, int(args.get("adults", 1)))),
                    children=max(0, min(9, int(args.get("children", 0)))),
                )
                print("search_flights tool called")
                content = json.dumps(out)
        elif name == "get_airports_or_cities":
            err = validate_tool_args_get_airports(args)
            if err:
                content = json.dumps({"error": err})
            else:
                out = amadeus.get_airports_or_cities(args.get("keyword", "").strip(), args.get("sub_type", "AIRPORT"))
                print("get_airports_or_cities tool called")
                content = json.dumps(out)
        elif name == "confirm_flight_price":
            raw = args.get("flight_offer_json", "{}")
            if len(raw) > MAX_TOOL_JSON_LENGTH:
                content = json.dumps({"error": "Flight offer payload too large."})
            else:
                try:
                    offer = json.loads(raw)
                    out = amadeus.confirm_flight_price(offer)
                except json.JSONDecodeError:
                    out = {"error": "Invalid flight offer JSON."}
                content = json.dumps(out)
        elif name == "create_flight_order":
            raw_offer = args.get("flight_offer_json", "{}")
            raw_travelers = args.get("travelers_json", "[]")
            if len(raw_offer) + len(raw_travelers) > MAX_TOOL_JSON_LENGTH:
                content = json.dumps({"error": "Payload too large."})
            else:
                try:
                    offer = json.loads(raw_offer)
                    travelers = json.loads(raw_travelers)
                    if not isinstance(travelers, list) or len(travelers) > 9:
                        out = {"error": "travelers must be a list of at most 9."}
                    else:
                        out = amadeus.create_flight_order(offer, travelers)
                except json.JSONDecodeError as e:
                    out = {"error": f"Invalid JSON: {e}"}
                content = json.dumps(out)
        else:
            content = json.dumps({"error": f"Unknown tool: {name}"})
        responses.append({"role": "tool", "content": content, "tool_call_id": tc.id})
    return responses

In [ ]:
# System prompt and agent run
SYSTEM_PROMPT = """You are a friendly flight booking attendant. You help users find and book flights using the Amadeus API.
Speak concisely and clearly. When you have flight options or a booking confirmation, state the key details (route, date, price, booking reference) in short sentences suitable for voice.
If the user says a city name, use get_airports_or_cities first to get IATA codes, then search_flights. Always confirm the price with confirm_flight_price before suggesting to book. For booking use create_flight_order with the offer and traveler details. If something fails, give a short voice-friendly message and suggest trying again or rephrasing."""

def agent_run(user_text, history):
    user_text = sanitize_user_input(user_text)
    if not user_text:
        return "Please provide a message."
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    history = (history or [])[-MAX_HISTORY_TURNS:]
    for h in history:
        messages.append({"role": h["role"], "content": (h.get("content") or "")[:MAX_USER_MESSAGE_LENGTH]})
    messages.append({"role": "user", "content": user_text})
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=TOOLS)
    while response.choices[0].finish_reason == "tool_calls":
        msg = response.choices[0].message
        messages.append(msg)
        for resp in handle_tool_calls(msg):
            messages.append(resp)
        response = client.chat.completions.create(model=MODEL, messages=messages, tools=TOOLS)
    return response.choices[0].message.content or ""

In [ ]:
# Audio: talker (text -> speech, like day5) — single function for reply audio
def talker(message):
    """Convert text to speech; returns audio bytes (Gradio can use path from a temp file)."""
    if not message or not str(message).strip():
        return None
    try:
        response = client.audio.speech.create(
            model="gpt-5-mini",
            voice="nova",
            input=str(message)[:4096]
        )
        fd, path = tempfile.mkstemp(suffix=".mp3")
        os.close(fd)
        with open(path, "wb") as f:
            f.write(response.content)
        return path
    except Exception as e:
        print(f"Talker error: {e}")
        return None

In [ ]:
# Gradio UI: text or voice input, chat + audio reply
def chat_with_agent(message, history):
    if not _rate_limit_check():
        return (history or []) + [{"role": "user", "content": str(message)[:200]}, {"role": "assistant", "content": "Too many requests. Please wait a minute and try again."}], None
    text = sanitize_user_input(message)
    if not text:
        return history or [], None
    history = list(history or []) + [{"role": "user", "content": text}]
    reply = agent_run(text, history[:-1])
    history = history + [{"role": "assistant", "content": reply}]
    audio_path = talker(reply) if reply else None
    return history, audio_path

with gr.Blocks(title="Flight Booking Voice Agent") as demo:
    gr.Markdown("## Flight Booking Voice Agent\nType your request. You get a text reply in the chat and an audio version below.")
    chatbot = gr.Chatbot(height=400, type="messages", label="Chat")
    msg = gr.Textbox(placeholder="Type your request...", label="Message")
    send_btn = gr.Button("Send")
    audio_output = gr.Audio(label="Assistant reply (audio)", autoplay=True)
    clear_btn = gr.Button("Clear")

    def clear():
        return [], None

    send_btn.click(chat_with_agent, inputs=[msg, chatbot], outputs=[chatbot, audio_output]).then(lambda: "", None, msg)
    msg.submit(chat_with_agent, inputs=[msg, chatbot], outputs=[chatbot, audio_output]).then(lambda: "", None, msg)
    clear_btn.click(clear, inputs=None, outputs=[chatbot, audio_output])

demo.launch(share=False, inbrowser=True)